# DATA PREPROCESSING

In [1]:
import numpy as np
import pandas as pd

In [2]:
DIR = '/home/014491542/CMPE-256 Project/Raw Data/'

#**Import CSV to dataframes**

In [3]:
price_datatype = {'store_id':'category','item_id':'category','wm_yr_wk':'int16','sell_price':'float32'}
df_prices = pd.read_csv(DIR+'sell_prices.csv',dtype=price_datatype)

In [4]:
for col, col_dtype in price_datatype.items():
        if col_dtype == "category":
            df_prices[col] = df_prices[col].cat.codes.astype("int16")
            df_prices[col] -= df_prices[col].min()

In [5]:
df_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,0,0,11325,9.58
1,0,0,11326,9.58
2,0,0,11327,8.26
3,0,0,11328,8.26
4,0,0,11329,8.26


In [6]:
cal_datatypes={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
df_calender = pd.read_csv(DIR+'calendar.csv',dtype=cal_datatypes)

In [7]:
for col, col_dtype in cal_datatypes.items():
        if col_dtype == "category":
            df_calender[col] = df_calender[col].cat.codes.astype("int16")
            df_calender[col] -= df_calender[col].min()

In [8]:
numcols = [f"d_{day}" for day in range(1,1942)]
catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']

val_datatype = {col:'float32' for col in numcols}
val_datatype.update({col:'category' for col in catcols})
df_eval = pd.read_csv(DIR+'sales_train_evaluation.csv',usecols= numcols + catcols, dtype=val_datatype)

In [9]:
df_eval.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0


In [10]:
for col in catcols:
        if col != "id":
            df_eval[col] = df_eval[col].cat.codes.astype("int16")
            df_eval[col] -= df_eval[col].min()

# **Pivot days columns to a single Sales columns**

In [11]:
df_eval = pd.melt(df_eval,
                  id_vars = catcols,
                  value_vars = [col for col in df_eval.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")

In [12]:
df_eval.shape

(59181090, 8)

# **Merge all Dataframes**

In [13]:
df = df_eval.merge(df_calender, on= "d", copy = False)
df = df.merge(df_prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)

In [14]:
df.shape

(46881677, 22)

# **Time Series Feature Creation**

In [15]:
# LAG COLUMNS FOR PAST 7,14 AND 28 DAYS

df['lag_7'] = df[['id','sales']].groupby('id')['sales'].shift(7)
df['lag_14'] = df[['id','sales']].groupby('id')['sales'].shift(14)
df['lag_21'] = df[['id','sales']].groupby('id')['sales'].shift(21)
df['lag_28'] = df[['id','sales']].groupby('id')['sales'].shift(28)

In [16]:
# ROLLING AVERAGE FOR PAST 7,14 AND 28 DAYS

df['rolling_ave_7_7'] = df[['id','lag_7']].groupby('id')['lag_7'].transform(lambda x : x.rolling(7).mean())
df['rolling_ave_14_14'] = df[['id','lag_14']].groupby('id')['lag_14'].transform(lambda x : x.rolling(14).mean())
df['rolling_ave_21_21'] = df[['id','lag_21']].groupby('id')['lag_21'].transform(lambda x : x.rolling(21).mean())
df['rolling_ave_28_28'] = df[['id','lag_28']].groupby('id')['lag_28'].transform(lambda x : x.rolling(28).mean())

# **Date Feature Creation**

In [17]:
df['date'] = pd.to_datetime(df['date'])
df['quarter'] = getattr(df['date'].dt,'quarter').astype('int16')
df['year'] = getattr(df['date'].dt,'year').astype('int16')
df['mday'] = getattr(df['date'].dt,'day').astype('int16')
df['is_month_start'] = getattr(df['date'].dt,'is_month_start').astype('int16')
df['is_month_end'] = getattr(df['date'].dt,'is_month_end').astype('int16')

In [18]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,lag_21,lag_28,rolling_ave_7_7,rolling_ave_14_14,rolling_ave_21_21,rolling_ave_28_28,quarter,mday,is_month_start,is_month_end
0,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_1,12.0,2011-01-29,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,1,29,0,0
1,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_2,15.0,2011-01-30,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,1,30,0,0
2,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_3,0.0,2011-01-31,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,1,31,0,1
3,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_4,0.0,2011-02-01,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,0
4,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_5,0.0,2011-02-02,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2,0,0


In [23]:
df_val = df[df['date']>'2016-04-24']
df_eval = df[df['date']<='2016-04-24']

In [25]:
df_val.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,lag_21,lag_28,rolling_ave_7_7,rolling_ave_14_14,rolling_ave_21_21,rolling_ave_28_28,quarter,mday,is_month_start,is_month_end
45966979,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,d_1914,0.0,2016-04-25,11613,...,0.0,1.0,0.857143,0.928571,0.952381,1.035714,2,25,0,0
45966980,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,d_1915,0.0,2016-04-26,11613,...,4.0,0.0,1.000000,0.928571,1.095238,0.964286,2,26,0,0
45966981,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,d_1916,0.0,2016-04-27,11613,...,2.0,0.0,1.142857,0.928571,1.142857,0.964286,2,27,0,0
45966982,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,d_1917,2.0,2016-04-28,11613,...,3.0,0.0,1.428571,1.000000,1.238095,0.928571,2,28,0,0
45966983,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,d_1918,0.0,2016-04-29,11613,...,0.0,0.0,1.285714,1.071429,1.095238,0.892857,2,29,0,0


# **Dump to CSV file**

In [26]:
df_val.to_csv('/home/014491542/CMPE-256 Project/DataSet/validationDataset.csv')

In [30]:
df_eval.to_csv('/home/014491542/CMPE-256 Project/DataSet/finalDataset.csv')